In [2]:
# region imports
from IPython import get_ipython

# noinspection PyBroadException
try:
    _magic = get_ipython().run_line_magic
    _magic("load_ext", "autoreload")
    _magic("autoreload", "2")
except:
    pass

# import datetime as dt
# import glob
import os
# import pprint
# import sys
# import matplotlib as mpl
# import matplotlib.colors
import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import seaborn as sns
import xarray as xr

# import bnn_tools.bnn_array

In [3]:
import nais_netcdf
import nais_netcdf.bnn_array
import nais_netcdf.funs as fu
import nais_netcdf.constants as co

In [4]:

DP = co.DP

DNDLDP = co.DNDLDP

TIME = co.TIME

FLAGS = co.FLAGS

MODE = co.MODE

POL_DIC = co.POL_DIC

MODE_DIC = co.MODE_DIC
P_ION = co.P_ION

MODES = co.MODES

META = co.META

In [5]:
# def main():
    # pass

In [6]:
data_path_in = os.path.join(nais_netcdf.__path__[0], 'data_in')
data_path_out = os.path.join(nais_netcdf.__path__[0], 'data_out')

In [7]:
log_dy = .025
sec_resample = 300
day = '20220720'

ds_comb = fu.get_full_ds(data_path_in, day, log_dy, sec_resample)

fu.save_ds2nc(ds_comb, data_path_out, day)

In [8]:
# this is what is inside get_full_ds

flag_ds = fu.get_flag_id_ds(data_path_in, day)

dss = []
for mode in MODES:
    dndldp_df = fu.get_mode_ds(
        data_path_in, day, log_dy, mode, sec_resample)
    dss.append(dndldp_df)

ds_comb:xr.Dataset = (
    xr.concat(dss, dim=MODE)
    .pipe(lambda d: xr.merge([d,flag_ds]))
)

for k,v in META.items():
    ds_comb[k] = ds_comb[k].assign_attrs(v)
